### Merging all paquet files at start of job...


In [1]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession


def loadMongoRDD(spark):
    '''
    Download data from mongodb and store it in RDD format
    '''

    dataRDD = spark.read.format("mongo") \
        .option('uri', f"mongodb://10.4.41.48/project3.new_data") \
        .load() \
        .rdd \
        .cache()

    return dataRDD

In [2]:
spark = SparkSession \
    .builder \
    .master(f"local[*]") \
    .appName("myApp") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

rdd = loadMongoRDD(spark)

22/06/02 22:55:27 WARN Utils: Your hostname, m1Mac-5.local resolves to a loopback address: 127.0.0.1; using 192.168.0.158 instead (on interface en0)
22/06/02 22:55:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/efwerr/.ivy2/cache
The jars for the packages stored in: /Users/efwerr/.ivy2/jars
:: loading settings :: url = jar:file:/Users/efwerr/miniforge3/envs/bdm_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cd3c5e24-ae8a-421c-ad45-a89f54d4bf0f;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 166ms :: artifacts dl 4ms

In [10]:
rdd.take(1)

[Row(District='Nou Barris', Info Idealista=[Row(bedrooms=1, date='2020_02_16', floor=5, latitude=41.4581612, longitude=2.1672375, operation='sale', price=115000.0, propertyCode=92912725, propertyType='flat', rooms=3, size=64.0), Row(bedrooms=1, date='2020_08_06', floor=None, latitude=41.456459, longitude=2.1863035, operation='sale', price=110000.0, propertyCode=38601752, propertyType='chalet', rooms=3, size=132.0), Row(bedrooms=1, date='2020_02_16', floor=5, latitude=41.4596697, longitude=2.170594, operation='sale', price=115000.0, propertyCode=92928008, propertyType='penthouse', rooms=3, size=67.0), Row(bedrooms=1, date='2020_08_06', floor=6, latitude=41.4642098, longitude=2.175942, operation='sale', price=129000.0, propertyCode=93070647, propertyType='flat', rooms=3, size=63.0)], Monthly Price (€/month)=525.7, Monthly Price Increase=-9.55, Neighborhood='Ciutat Meridiana', Population Increase=314.0, Population recent=10369.0, RFD Increased=4.3, RFD most recent=38.6, Surface Price (€/m

In [136]:
from pyspark.mllib.regression import LabeledPoint

newRDD = rdd.flatMap(lambda x: [(i[7], i[8], i[5], i[6], i[9], i[0], i[10], i[2], i[3], i[4], i[1],\
                            x[0], x[4], x[2], x[5], x[6], x[7], x[8], x[9], x[10]) for i in x[1]])

labelRDD = newRDD.map(lambda x: LabeledPoint(x[3], [x[4], x[5], x[6], x[7], x[13], x[14], x[15], x[16], x[17], x[18], x[19]]))
#labelRDD = newRDD.map(lambda x: LabeledPoint(x[3], [float(x[i]) for i in range(1,20) if i != 3]))

3905

In [170]:
labelRDD.take(1)

[LabeledPoint(115000.0, [3.0,1.0,64.0,5.0,525.7,314.0,10369.0,4.3,38.6,8.9,-0.22])]

In [169]:
newRDD.count()

3905

In [140]:
newRDD.take(1)

[(92912725,
  'flat',
  'sale',
  115000.0,
  3,
  1,
  64.0,
  5,
  41.4581612,
  2.1672375,
  '2020_02_16',
  'Nou Barris',
  'Ciutat Meridiana',
  525.7,
  314.0,
  10369.0,
  4.3,
  38.6,
  8.9,
  -0.22)]

In [167]:
from pyspark.mllib.regression import LabeledPoint

labelRDD = newRDD.map(lambda x: LabeledPoint(x[3], [x[4], x[5], x[6], x[7], x[13], x[14], x[15], x[16], x[17], x[18], x[19]]))
newRDD.count()

3905

In [201]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.util import MLUtils

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = labelRDD.randomSplit([0.7, 0.3], seed=42)

# Train a RandomForest model.
model = RandomForest.trainRegressor(trainingData, categoricalFeaturesInfo={},
                                    numTrees=5, maxDepth=5, maxBins=64, seed=42)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression forest model:')
print(model.toDebugString())


Test Mean Squared Error = 160164064423.47583
Learned regression forest model:
TreeEnsembleModel regressor with 5 trees

  Tree 0:
    If (feature 2 <= 231.0)
     If (feature 4 <= 1044.625)
      If (feature 2 <= 113.5)
       If (feature 1 <= 1.5)
        If (feature 4 <= 890.55)
         Predict: 225302.49328449328
        Else (feature 4 > 890.55)
         Predict: 315711.10062893084
       Else (feature 1 > 1.5)
        If (feature 8 <= 103.0)
         Predict: 336042.104
        Else (feature 8 > 103.0)
         Predict: 464167.08754208754
      Else (feature 2 > 113.5)
       If (feature 4 <= 937.525)
        If (feature 2 <= 150.5)
         Predict: 464823.5294117647
        Else (feature 2 > 150.5)
         Predict: 676000.0
       Else (feature 4 > 937.525)
        If (feature 3 <= 3.5)
         Predict: 651550.0
        Else (feature 3 > 3.5)
         Predict: 836723.8805970149
     Else (feature 4 > 1044.625)
      If (feature 1 <= 1.5)
       If (feature 4 <= 1204.034999999

22/06/03 04:30:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 515736 ms exceeds timeout 120000 ms
22/06/03 04:30:04 WARN SparkContext: Killing executors is not supported by current scheduler.
